In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, solve_discrete_lyapunov
from mpl_toolkits.axes_grid1 import make_axes_locatable

def build_ssm_approximation(kernel, delta_k, N):
    """
    Построение SSM, аппроксимирующей ядро kernel на сетке с шагом delta_k
    """
    # Дискретизация ядра
    T = np.arange(0, 10*delta_k, delta_k)
    Y = np.array([kernel(t, t) for t in T])
    
    # Реализация в пространстве состояний (фикс размерностей)
    A = np.diag(np.exp(-np.linspace(0.1, 1.0, N)))
    C = np.ones((1, N))  # (1, N)
    
    # Решение уравнения наблюдения (фикс размерности B)
    P = solve_discrete_lyapunov(A, np.eye(N))
    B = np.linalg.solve(P, C.T)  # (N, 1) вместо (1, N)
    
    # Подгонка под целевую функцию (фикс матричных операций)
    Gamma = np.array([(C @ np.linalg.matrix_power(A, j) @ B).item() for j in range(len(T))])
    beta = np.linalg.lstsq(Gamma.reshape(-1, 1), Y.reshape(-1, 1), rcond=None)[0]
    
    return A, B * beta, C

def universal_approximator(attn_block, L, epsilon):
    """
    Построение ансамбля SSM для аппроксимации блока внимания
    """
    # Определение числа голов
    K = int(np.ceil(1.0 / epsilon))
    delta = L / K
    
    models = []
    weights = []
    
    for k in range(1, K+1):
        # Построение SSM для k-ой подпоследовательности
        A, B, C = build_ssm_approximation(
            lambda t, s: attn_block.kernel(t, s),
            delta_k = k * delta,
            N = int(np.ceil(np.log(1/epsilon))))
        
        models.append((A, B, C))
        weights.append(1.0 / k)
    
    # Нормализация весов
    weights = np.array(weights) / np.sum(weights)
    
    return models, weights, K

def visualize_kernels(attn_block, models, weights, L, K):
    """
    Визуализация сравнения ядер внимания и SSM-аппроксимации
    """
    # Создание сетки
    t = np.linspace(0, L, 100)
    s = np.linspace(0, L, 100)
    T, S = np.meshgrid(t, s)
    
    # Истинное ядро
    true_kernel = np.vectorize(attn_block.kernel)(T, S)
    
    # Аппроксимированное ядро
    approx_kernel = np.zeros_like(T)
    for idx, (A, B, C) in enumerate(models):
        for i in range(len(t)):
            for j in range(len(s)):
                delay = np.abs(t[i] - s[j])
                steps = int(delay // (L / K))
                if steps < 100:
                    # Матричное умножение с правильными размерностями
                    power = np.linalg.matrix_power(A, steps)
                    approx_kernel[i, j] += weights[idx] * (C @ power @ B).item()
    
    # Разность
    diff = np.abs(true_kernel - approx_kernel)
    
    # Визуализация
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    titles = [
        "Трансформер: Истинное ядро",
        f"Mamba: Аппроксимация (K={K})",
        f"Разность (MSE: {np.mean(diff**2):.2e})"
    ]
    
    for i, data in enumerate([true_kernel, approx_kernel, diff]):
        im = axs[i].imshow(data, cmap='viridis' if i < 2 else 'plasma', 
                          origin='lower', extent=[0, L, 0, L])
        axs[i].set_title(titles[i])
        axs[i].set_xlabel("Позиция s")
        axs[i].set_ylabel("Позиция t")
        fig.colorbar(im, ax=axs[i])
    
    plt.tight_layout()
    plt.savefig(f"kernel_approx_K{K}.png", dpi=120)
    plt.close()
    
    return true_kernel, approx_kernel

def plot_convergence(attn_block, L, max_K=20):
    """
    Анализ сходимости при увеличении числа голов
    """
    errors = []
    K_values = range(1, max_K+1)
    
    for K in K_values:
        _, approx_kernel = visualize_kernels(
            attn_block,
            *universal_approximator(attn_block, L, 1/K)[:2],
            L, K
        )
        # Вычисление ошибки
        diff = np.abs(attn_block.kernel(0.3*L, 0.7*L) - approx_kernel[int(30), int(70)])
        errors.append(diff)
    
    # График
    plt.figure(figsize=(10, 6))
    plt.plot(K_values, errors, 'o-', label="Эксперимент")
    plt.plot(K_values, 1/np.array(K_values), 'r--', label="Теория O(1/K)")
    plt.xlabel("Число голов (K)")
    plt.ylabel("Ошибка аппроксимации")
    plt.title("Сходимость к трансформеру")
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.grid(True)
    plt.savefig("convergence.png", dpi=120)
    plt.close()
    
    return errors

def visualize_head_responses(models, weights, L, K):
    """
    Визуализация импульсных характеристик голов
    """
    plt.figure(figsize=(12, 8))
    
    for idx, (A, B, C) in enumerate(models):
        # Импульсная характеристика
        response = []
        h = np.zeros((A.shape[0], 1))
        impulse = np.zeros(100)
        impulse[0] = 1.0
        
        for i in range(100):
            h = A @ h + B * impulse[i]
            response.append((C @ h).item())
        
        # График
        plt.plot(response, lw=2, 
                label=f"Голова {idx+1} (шаг={K/(idx+1):.1f}, вес={weights[idx]:.3f})")
    
    plt.title("Импульсные характеристики SSM-голов")
    plt.xlabel("Временной шаг")
    plt.ylabel("Отклик")
    plt.grid(True)
    plt.legend()
    plt.savefig("head_responses.png", dpi=120)
    plt.close()

# Тестовый блок внимания
class AttentionBlock:
    def __init__(self, kernel_type='gaussian'):
        self.kernel_type = kernel_type
        
    def kernel(self, t, s):
        if self.kernel_type == 'gaussian':
            return np.exp(-0.1*(t-s)**2)
        elif self.kernel_type == 'laplace':
            return np.exp(-0.2*np.abs(t-s))
        else:  # constant
            return 1.0 if np.abs(t-s) < 0.1*L else 0.0

# Пример использования
if __name__ == "__main__":
    L = 100  # Длина последовательности
    attn_block = AttentionBlock(kernel_type='gaussian')
    
    # Построение аппроксиматора
    models, weights, K = universal_approximator(attn_block, L, 0.2)
    
    # Визуализация
    visualize_kernels(attn_block, models, weights, L, K)
    plot_convergence(attn_block, L)
    visualize_head_responses(models, weights, L, K)